# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [4]:
%%configure
{
    "--job-bookmark-option":"job-bookmark-enable"
}

The following configurations have been updated: {'--job-bookmark-option': 'job-bookmark-enable'}


In [1]:
# # SPARK

# # defining the resource constants
# bucket_name = "data-engineering-project-920372994009"
# source_folder = "bronze_data"
# processed_folder = "silver_data"
# db_name = "dev"
# table_name = "Product"

# # reading the data using inferring schema
# df = spark.read.csv(f"s3://{bucket_name}/{source_folder}/{db_name}/{table_name}", sep = ",", header=True, inferSchema=True)
# df.printSchema()
# df.show()

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.8 


In [2]:
# from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# product_schema = StructType(
#     fields = (
#         [
#             StructField("op", StringType()),
#             StructField("productId", StringType()),
#             StructField("productName", StringType()),
#             StructField("brandName", StringType()),
#             StructField("productDescription", StringType()),
#             StructField("price", DoubleType()),
#             StructField("productCategory", StringType()),
#         ]
#     )
# )
# # reading data from a structType(manually)
# products_df = spark.read.option("header","true").option("delimeter",",").schema(product_schema).csv(f"s3://{bucket_name}/{source_folder}/{db_name}/{table_name}/")
# products_df.printSchema()
# products_df.show()

In [2]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql.functions import current_timestamp, current_date, lit, concat_ws, col, sha2, round
from pyspark.sql.types import TimestampType
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from awsglue.utils import getResolvedOptions

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
args = getResolvedOptions(sys.argv,['JOB_NAME'])
job.init(args['JOB_NAME'], args)

# defining glue resource constants
glue_database = "data-engineering-project-glue-db"
glue_table = "raw_data_product"

# reading data from a glue data catalogue
products_df_from_catalog = glueContext.create_data_frame_from_catalog(glue_database,glue_table,additional_options = {"useCatalogueSchema":True}, transformation_ctx = 'products_df_from_catalog')

# renaming the columns for silver_data folder
# from pyspark.sql.dataframe import withColumnRenamed
if products_df_from_catalog.count() > 0:
    renamed_products = products_df_from_catalog.withColumnRenamed("op","cdc_operations")\
                              .withColumnRenamed("productid","product_id")\
                              .withColumnRenamed("productname","product_name")\
                              .withColumnRenamed("brandname","brand_name")\
                              .withColumnRenamed("productdescription","product_description")\
                              .withColumnRenamed("price","product_price")\
                              .withColumnRenamed("productcategory","product_category")

    # apply the slowly changing dimension - 2 here , i.e, using versions/active_flag/date to keep track of changing dimensions
    current_timestamp = current_timestamp()
    current_date = current_date()
    record_end_ts = lit('9999-12-31').cast(TimestampType())
    active_flag = lit(1)

    # here, we'll be using "sha2" function to create a hash_value for each record being updated, 
    # hash_value is basically the encrypted(concatenated values of a row)

    hash_value = concat_ws('',col("product_id"),col("product_name"),col("brand_name"),col("product_description"),col("product_price"),col("product_category"))

    # final_df with scd fields

    final_product_df = renamed_products.withColumn("hash_value", sha2(hash_value, 256))\
                                       .withColumn("record_start_ts", current_timestamp)\
                                       .withColumn("record_end_ts", record_end_ts)\
                                       .withColumn("ingestion_date", current_date)\
                                       .withColumn("active_flag", active_flag)\
                                       .withColumn("product_price", round(col("product_price"),2))

    # now we'll creating a dynamicframe from this dataframe and store it into processed folder(S3) in parquet format.
    final_product_dyf = DynamicFrame.fromDF(final_product_df, glueContext, "final_product_dyf")

    source_bucket = "data-engineering-project-920372994009"
    processed_folder = "silver_data"
    db_name = "dev"
    table_name = "Product"

    glueContext.write_dynamic_frame.from_options(
        frame = final_product_dyf,
        connection_type = "s3",
        connection_options = {"path": f"s3://{source_bucket}/{processed_folder}/{db_name}/{table_name}/", "partitionKeys" : ["ingestion_date"]},
        format = 'parquet',
        transformation_ctx = 'products_dyf_to_s3'
    )
else:
    print(f"no data found in {glue_database}.{glue_table}")
        

job.commit()